In [ ]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [ ]:
FILE = open('/content/drive/MyDrive/Colab Notebooks/NLP_labs/sentences.txt')
SENTENCES = FILE.read().split('. ')[:-1]

max_tokens = 10000
tokens = 0

for sent in SENTENCES:
  tokens+=len(sent.split())
avg_tokens = round(tokens/len(sentences))

avg_tokens
SENTENCES

['which input is most helpful is classifying data without error? These input-weight products are summed and then the sum is passed through a node’s so-called activation function, to determine whether and to what extent that signal should progress further through the network to affect the ultimate outcome, say, an act of classification',
 'And thanks to the demand from gamers around the world, GPUs (graphics processing units) make it possible for us to leverage deep learning algorithms to build and train models with impressive results in a time-efficient manner! So to all the parents who don’t like your kids playing games: Gaming has its silver lining…',
 'Is the dataset you need publicly available, or can you create it (with a data annotation service like Scale or AWS Mechanical Turk)? In this example, spam emails would be labeled as spam, and the labels would enable the algorithm to map from inputs to the classifications you care about',
 '(Neural networks can also extract features th

In [ ]:
targets = []

for sent in SENTENCES:
  targets.append({'Sentence':sent, 'Target':'Deep learning'})

targets = pd.DataFrame(data = targets, columns=['Sentence', 'Target'])

targets['Label'] = targets['Target']
targets.replace({'Label':{'Deep learning':0, 'Neural network':1}}, inplace=True)
targets

,Sentence,Target,Label
0,which input is most helpful is classifying dat...,Deep learning,0
1,And thanks to the demand from gamers around th...,Deep learning,0
2,"Is the dataset you need publicly available, or...",Deep learning,0
3,(Neural networks can also extract features tha...,Deep learning,0
4,Deep learning algorithms have improved over th...,Deep learning,0
...,...,...,...
94,That’s why deep learning is important,Deep learning,0
95,Deep learning maps inputs to outputs,Deep learning,0
96,This is known as supervised learning,Deep learning,0
97,Learn to build AI in simulations »,Deep learning,0


#### Tokenization&Embedding

In [ ]:
text_vectorization = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorization.adapt(targets['Sentence'])

text_vectorization(targets['Sentence'])

<tf.Tensor: shape=(99, 18), dtype=int64, numpy=
array([[ 26,  36,   6, ...,   5,  69,   2],
       [  5, 302,   3, ...,  16, 168,  18],
       [  6,   2, 123, ..., 659, 347,  34],
       ...,
       [ 43,   6, 193, ...,   0,   0,   0],
       [ 58,   3, 127, ...,   0,   0,   0],
       [167, 664, 368, ...,   0,   0,   0]])>

In [ ]:
embedding = layers.Embedding(input_dim=max_tokens, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=avg_tokens) 

embedding(text_vectorization(targets['Sentence']))

<tf.Tensor: shape=(99, 18, 128), dtype=float32, numpy=
array([[[-5.14935330e-03,  2.00499408e-02, -9.40989330e-03, ...,
         -4.86432388e-03, -3.76358405e-02,  1.77688934e-02],
        [-5.33986092e-03,  4.98909689e-02,  1.78847425e-02, ...,
         -2.90019996e-02,  2.00910605e-02, -1.72149539e-02],
        [-3.88250127e-02,  1.15276203e-02, -2.82633305e-03, ...,
          1.34248771e-02, -3.48344222e-02, -1.82900429e-02],
        ...,
        [ 4.63610627e-02,  2.76350714e-02,  4.24636267e-02, ...,
         -4.54208255e-02, -4.73252051e-02,  4.51025255e-02],
        [-4.07841429e-02,  1.78874470e-02,  2.74449475e-02, ...,
         -3.13869603e-02, -4.12852280e-02, -1.81551464e-02],
        [-3.70614752e-02, -2.15432644e-02,  1.04018673e-02, ...,
          4.70650680e-02, -2.82889362e-02,  1.74922012e-02]],

       [[ 4.63610627e-02,  2.76350714e-02,  4.24636267e-02, ...,
         -4.54208255e-02, -4.73252051e-02,  4.51025255e-02],
        [-4.27822247e-02, -3.96378636e-02,  5.51

#### Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(targets, train_size=0.8, test_size=0.2)
train_split

,Sentence,Target,Label
95,Deep learning maps inputs to outputs,Deep learning,0
74,Deep learning does not require labels to detec...,Deep learning,0
63,"Search: Comparing documents, images or sounds ...",Deep learning,0
90,"Employee turnover (ditto, but for employees)",Deep learning,0
4,Deep learning algorithms have improved over th...,Deep learning,0
...,...,...,...
47,An artificial neural network represents the st...,Deep learning,0
80,Learning without labels is called unsupervised...,Deep learning,0
52,"As you can see, with neural networks, we’re mo...",Deep learning,0
64,"Also, this “oil refinery” is improving on both...",Deep learning,0


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: Naive Bayes Model, Dense Neural Network

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_NB = Pipeline([
                    ("tfidf", TfidfVectorizer()), 
                    ("clf", MultinomialNB()) 
])

# Fit the pipeline to the training data
model_NB.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_NB.score(test_split['Sentence'], test_split['Label'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 100.00%


### LSTM

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorization(inputs)
x = embedding(x)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_lstm = tf.keras.Model(inputs, outputs, name="model_LSTM")


In [ ]:
model_lstm.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_lstm.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 18)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 18, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                        

In [ ]:
model_lstm_history = model_lstm.fit(train_split['Sentence'], train_split['Label'], epochs=4, validation_data=(test_split['Sentence'], test_split['Label']))

Epoch 1/4
3/3 [==============================] - 7s 363ms/step - loss: 0.6748 - accuracy: 0.9747 - val_loss: 0.6409 - val_accuracy: 1.0000
Epoch 2/4
3/3 [==============================] - 0s 27ms/step - loss: 0.6332 - accuracy: 1.0000 - val_loss: 0.5904 - val_accuracy: 1.0000
Epoch 3/4
3/3 [==============================] - 0s 29ms/step - loss: 0.5788 - accuracy: 1.0000 - val_loss: 0.5183 - val_accuracy: 1.0000
Epoch 4/4
3/3 [==============================] - 0s 32ms/step - loss: 0.5010 - accuracy: 1.0000 - val_loss: 0.4137 - val_accuracy: 1.0000


In [ ]:
model_lstm_pred_probs = model_lstm.predict(test_split['Sentence'])
model_lstm_preds = tf.squeeze(tf.round(model_lstm_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_lstm_preds)


{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}

### GRU

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorization(inputs)
x = embedding(x)
x = layers.GRU(64)(x) 
x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_gru = tf.keras.Model(inputs, outputs, name="model_GRU")


In [ ]:
model_gru.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [ ]:
model_gru_history = model_gru.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=5,
                              validation_data=(test_split['Sentence'], test_split['Label']))


Epoch 1/5
3/3 [==============================] - 3s 306ms/step - loss: 0.6967 - accuracy: 0.3924 - val_loss: 0.6571 - val_accuracy: 1.0000
Epoch 2/5
3/3 [==============================] - 0s 26ms/step - loss: 0.6411 - accuracy: 1.0000 - val_loss: 0.6039 - val_accuracy: 1.0000
Epoch 3/5
3/3 [==============================] - 0s 29ms/step - loss: 0.5808 - accuracy: 1.0000 - val_loss: 0.5277 - val_accuracy: 1.0000
Epoch 4/5
3/3 [==============================] - 0s 27ms/step - loss: 0.4978 - accuracy: 1.0000 - val_loss: 0.4276 - val_accuracy: 1.0000
Epoch 5/5
3/3 [==============================] - 0s 27ms/step - loss: 0.3909 - accuracy: 1.0000 - val_loss: 0.3095 - val_accuracy: 1.0000


In [ ]:
model_gru_pred_probs = model_gru.predict(test_split['Sentence'])
model_gru_preds = tf.squeeze(tf.round(model_gru_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_gru_preds)

{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}

### Bidirectional LSTM

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorization(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # stacking RNN layers requires return_sequences=True
x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_lstm = tf.keras.Model(inputs, outputs, name="model_Bi_lstm")


In [ ]:
model_bi_lstm.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [ ]:
model_bi_lstm.summary()


Model: "model_Bi_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 18)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 18, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
N

In [ ]:
model_bi_lstm_history = model_bi_lstm.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=4,
                              validation_data=(test_split['Sentence'], test_split['Label']))


Epoch 1/4
3/3 [==============================] - 6s 583ms/step - loss: 0.6752 - accuracy: 0.7468 - val_loss: 0.6207 - val_accuracy: 1.0000
Epoch 2/4
3/3 [==============================] - 0s 32ms/step - loss: 0.5972 - accuracy: 1.0000 - val_loss: 0.5355 - val_accuracy: 1.0000
Epoch 3/4
3/3 [==============================] - 0s 33ms/step - loss: 0.5036 - accuracy: 1.0000 - val_loss: 0.4222 - val_accuracy: 1.0000
Epoch 4/4
3/3 [==============================] - 0s 34ms/step - loss: 0.3763 - accuracy: 1.0000 - val_loss: 0.2710 - val_accuracy: 1.0000


### Bidirectional GRU

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorization(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.GRU(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_bi_gru = tf.keras.Model(inputs, outputs, name="model_Bi_gru")

In [ ]:
model_bi_gru.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

In [ ]:
model_bi_gru.summary()

Model: "model_Bi_gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 18)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 18, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,354,625
Trainable params: 1,354,625
No

In [ ]:
model_bi_gru_history = model_bi_gru.fit(train_split['Sentence'],
                              train_split['Label'],
                              epochs=10,
                              validation_data=(test_split['Sentence'], test_split['Label']))

Epoch 1/10
3/3 [==============================] - 5s 520ms/step - loss: 0.6872 - accuracy: 0.5823 - val_loss: 0.6094 - val_accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 0s 32ms/step - loss: 0.5741 - accuracy: 1.0000 - val_loss: 0.5070 - val_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 0s 34ms/step - loss: 0.4671 - accuracy: 1.0000 - val_loss: 0.4036 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 0s 33ms/step - loss: 0.3577 - accuracy: 1.0000 - val_loss: 0.2972 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 0s 32ms/step - loss: 0.2476 - accuracy: 1.0000 - val_loss: 0.1923 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 0s 38ms/step - loss: 0.1450 - accuracy: 1.0000 - val_loss: 0.1004 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 36ms/step - loss: 0.0624 - accuracy: 1.0000 - val_loss: 0.0362 - val_accuracy: 1.0000
Epoch 8/10
3/3 [==========

## Здійснити порівняння ефективності моделей

### Naive Bayes

In [ ]:
calculate_results(test_split['Label'], model_NB.predict(test_split['Sentence']))

{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}

### LSTM

In [ ]:
model_lstm_pred_probs = model_lstm.predict(test_split['Sentence'])
model_lstm_preds = tf.squeeze(tf.round(model_lstm_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_lstm_preds)

{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}

### GRU

In [ ]:
model_gru_pred_probs = model_gru.predict(test_split['Sentence'])
model_gru_preds = tf.squeeze(tf.round(model_gru_pred_probs))
calculate_results(y_true=test_split['Label'], y_pred=model_gru_preds)

{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}

### BI_LSTM


In [ ]:
model_bi_lstm_pred_probs = model_bi_lstm.predict(test_split['Sentence'])
model_bi_lstm_preds = tf.squeeze(tf.round(model_bi_lstm_pred_probs))
calculate_results(test_split['Label'], model_bi_lstm_preds)

{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}

### BI_GRU

In [ ]:
model_bi_gru_pred_probs = model_bi_gru.predict(test_split['Sentence'])
model_bi_gru_preds = tf.squeeze(tf.round(model_bi_gru_pred_probs))
calculate_results(test_split['Label'], model_bi_gru_preds)

{'accuracy': 100.0, 'f1': 1.0, 'precision': 1.0, 'recall': 1.0}